In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re
import numpy as np
import pandas as pd

In [22]:
client_credentials_manager = SpotifyClientCredentials(client_id='cf5e39da3b50438493846ee960df886d',client_secret='23e7cc1dcb4f4836bc594667c6e2cef3')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [23]:
def track_features(trackId):
    features = sp.audio_features(trackId)[0]
    artist = sp.track(trackId)['artists'][0]['id']
    artist_pop = sp.artist(artist)['popularity']
    artist_genres =  sp.artist(artist)['genres']
    
    track_pop = sp.track(trackId)['popularity']
    
    features['artist_pop'] = artist_pop
    if artist_genres:
        features['genres'] = " ".join([re.sub(' ','_',i) for i in artist_genres])
    else:
        features['genres'] = 'unknown'
        
    features['track_pop'] = track_pop
    
    return features

In [24]:
playlist = sp.playlist('37i9dQZF1DXdOEFt9ZX0dh')

playlist_df = pd.DataFrame(columns=['artist_name','track_uri','artist_uri','track_name','album_uri','duration_ms','album_name'])

for i in range(0,len(playlist['tracks']['items'])):
    attribute_list=[]
    attribute_list.append(playlist['tracks']['items'][i]['track']['artists'][0]['name'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['uri'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['artists'][0]['uri'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['name'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['album']['uri'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['duration_ms'])
    attribute_list.append(playlist['tracks']['items'][i]['track']['album']['name'])
    playlist_df.loc[len(playlist_df)] = attribute_list

playlist_df['track_uri'] = playlist_df['track_uri'].apply(lambda x:x[x.rfind(':')+1:])

playlist_df.head()

,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,Creedence Clearwater Revival,6NxsCnLeLd8Ai1TrgGxzIx,spotify:artist:3IYUhFvPQItj6xySrBmZkd,Bad Moon Rising,spotify:album:30sgpxWf6U9417FySmLEGK,141600,Green River (40th Anniversary Edition)
1,Boston,1QEEqeFIZktqIpPI4jSVSF,spotify:artist:29kkCKKGXheHuoO829FxWK,More Than a Feeling,spotify:album:2QLp07RO6anZHmtcKTEvSC,285133,Boston
2,AC/DC,2SiXAy7TuUkycRVbbWDEpo,spotify:artist:711MCceyCBcFnzjGY4Q7Un,You Shook Me All Night Long,spotify:album:6mUdeDZCsExyJLMdAfDuwh,210173,Back In Black
3,Derek & The Dominos,2kkvB3RNRzwjFdGhaUA0tz,spotify:artist:2rc78XDH9zuJP6bm78lU8Z,Layla,spotify:album:5iIWnMgvSM8uEBwXKsPcXM,423840,Layla And Other Assorted Love Songs (Remastere...
4,The Black Crowes,05NYcsjJwOYq4jIiKPVj9p,spotify:artist:5krkohEVJYw0qoB5VWwxaC,Hard To Handle,spotify:album:2NRRQLuW6j3EsoWpIl2MR3,188066,Shake Your Money Maker


In [25]:
key_columns=[]
for key in track_features(playlist_df['track_uri'][0]):
    key_columns.append(key)

track_features_df = pd.DataFrame(columns=key_columns)

featureList = []
for i in range(0,len(playlist_df)):
#     print(i)
    featureList.append(track_features(playlist_df['track_uri'][i]))

track_features_df = pd.DataFrame(featureList)

track_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artist_pop,genres,track_pop
0,0.647,0.762,2,-5.856,1,0.0287,0.05990,0.000003,0.0705,0.930,...,audio_features,6NxsCnLeLd8Ai1TrgGxzIx,spotify:track:6NxsCnLeLd8Ai1TrgGxzIx,https://api.spotify.com/v1/tracks/6NxsCnLeLd8A...,https://api.spotify.com/v1/audio-analysis/6Nxs...,141600,4,80,album_rock classic_rock country_rock rock sout...,0
1,0.377,0.681,7,-8.039,1,0.0298,0.00088,0.002300,0.0504,0.285,...,audio_features,1QEEqeFIZktqIpPI4jSVSF,spotify:track:1QEEqeFIZktqIpPI4jSVSF,https://api.spotify.com/v1/tracks/1QEEqeFIZktq...,https://api.spotify.com/v1/audio-analysis/1QEE...,285133,4,70,album_rock blues_rock classic_rock country_roc...,80
2,0.532,0.767,7,-5.509,1,0.0574,0.00287,0.000513,0.3900,0.755,...,audio_features,2SiXAy7TuUkycRVbbWDEpo,spotify:track:2SiXAy7TuUkycRVbbWDEpo,https://api.spotify.com/v1/tracks/2SiXAy7TuUky...,https://api.spotify.com/v1/audio-analysis/2SiX...,210173,4,85,australian_rock hard_rock rock,81
3,0.404,0.902,1,-3.880,1,0.0665,0.57700,0.297000,0.2870,0.497,...,audio_features,2kkvB3RNRzwjFdGhaUA0tz,spotify:track:2kkvB3RNRzwjFdGhaUA0tz,https://api.spotify.com/v1/tracks/2kkvB3RNRzwj...,https://api.spotify.com/v1/audio-analysis/2kkv...,423840,4,64,album_rock art_rock blues blues_rock british_b...,77
4,0.599,0.790,4,-3.580,1,0.0430,0.00257,0.001810,0.3120,0.961,...,audio_features,05NYcsjJwOYq4jIiKPVj9p,spotify:track:05NYcsjJwOYq4jIiKPVj9p,https://api.spotify.com/v1/tracks/05NYcsjJwOYq...,https://api.spotify.com/v1/audio-analysis/05NY...,188067,4,65,album_rock alternative_rock blues_rock classic...,74


In [26]:
music_df = pd.merge(playlist_df,track_features_df,left_on='track_uri',right_on = 'id')

In [27]:
music_df = music_df.drop_duplicates()

In [28]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       100 non-null    object 
 1   track_uri         100 non-null    object 
 2   artist_uri        100 non-null    object 
 3   track_name        100 non-null    object 
 4   album_uri         100 non-null    object 
 5   duration_ms_x     100 non-null    int64  
 6   album_name        100 non-null    object 
 7   danceability      100 non-null    float64
 8   energy            100 non-null    float64
 9   key               100 non-null    int64  
 10  loudness          100 non-null    float64
 11  mode              100 non-null    int64  
 12  speechiness       100 non-null    float64
 13  acousticness      100 non-null    float64
 14  instrumentalness  100 non-null    float64
 15  liveness          100 non-null    float64
 16  valence           100 non-null    float64
 17

In [29]:
music_df.describe()

,duration_ms_x,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_y,time_signature,artist_pop,track_pop
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.0,100.000000,100.000000
mean,264565.190000,0.538790,0.740110,5.520000,-8.133020,0.800000,0.059318,0.147308,0.036131,0.176558,0.64982,125.433660,264565.510000,4.0,71.500000,59.170000
std,72250.650266,0.139019,0.160159,3.170189,3.094809,0.402015,0.050344,0.177332,0.100377,0.127295,0.21058,25.760827,72250.722723,0.0,7.057484,27.365116
min,141600.000000,0.257000,0.334000,0.000000,-18.294000,0.000000,0.027100,0.000146,0.000000,0.034100,0.15000,77.810000,141600.000000,4.0,57.000000,0.000000
25%,214903.250000,0.444750,0.629250,2.000000,-9.994000,1.000000,0.034250,0.010437,0.000022,0.083550,0.49700,108.573000,214903.250000,4.0,66.000000,64.000000
50%,262486.500000,0.545000,0.753500,5.500000,-7.656500,1.000000,0.042750,0.061950,0.000476,0.125500,0.67400,124.655500,262486.500000,4.0,70.500000,70.000000
75%,288253.250000,0.644250,0.873250,9.000000,-5.554000,1.000000,0.065150,0.251000,0.009262,0.260750,0.80450,140.099750,288253.250000,4.0,76.000000,75.000000
max,536226.000000,0.831000,0.987000,11.000000,-3.580000,1.000000,0.354000,0.780000,0.635000,0.709000,0.97800,189.394000,536227.000000,4.0,89.000000,83.000000


In [30]:
music_df.to_csv('C:\\Users\\Akhil_S_Bhat\\Documents\\AIML\Machine_Learning\\Unsupervised Learning\\Data\\3_Spotify_Data\\spotify_test_data.csv')